# **YOLOv3 Object Detection - mAP calculation**

In [1]:
#!/usr/bin/env python3

# YOLOv3
# * Trained on COCO 2017 dataset.
#
# 2021 (c) Nicole Hölzl
# 2021 (c) Micha Johannes Birklbauer
# 2021 (c) The TensorFlow Authors
#
# https://github.com/t0xic-m/
# micha.birklbauer@gmail.com

import os
import imageio
import numpy as np
from matplotlib import pyplot as plt

## **Testing on our test sample**

In [2]:
import pandas as pd

test_df = pd.read_csv("../data/results_yolo/test_human_pose_bbox.csv")
test_df.rename(columns={"img_names": "NAME", "up_left_x": "xmin_pred", "up_left_y": "ymin_pred", "low_right_x": "xmax_pred",
    "low_right_y": "ymax_pred"}, inplace=True)
test_df.head(5)

,NAME,xmin_pred,ymin_pred,xmax_pred,ymax_pred
0,071718689.jpg,717,300,1024,624
1,046808162.jpg,679,80,924,565
2,065158590.jpg,499,1,1187,597
3,007055006.jpg,278,102,431,265
4,020584121.jpg,229,9,425,459


In [3]:
ground_truth_df = pd.read_csv("../data/samples_test.csv")
ground_truth_df.head(5)

,NAME,ymin,ymax,xmin,xmax
0,002094496.jpg,266.0,1080.0,887.0,1686.0
1,007055006.jpg,100.0,274.0,282.0,429.0
2,007349675.jpg,226.0,437.0,87.0,361.0
3,011151221.jpg,82.0,349.0,239.0,429.0
4,018050712.jpg,147.0,984.0,573.0,1919.0


In [4]:
result = pd.merge(ground_truth_df,
                 test_df,
                 on='NAME')
result.head()

,NAME,ymin,ymax,xmin,xmax,xmin_pred,ymin_pred,xmax_pred,ymax_pred
0,002094496.jpg,266.0,1080.0,887.0,1686.0,912,278,1681,1025
1,007055006.jpg,100.0,274.0,282.0,429.0,278,102,431,265
2,007349675.jpg,226.0,437.0,87.0,361.0,100,219,368,438
3,011151221.jpg,82.0,349.0,239.0,429.0,222,83,389,346
4,018050712.jpg,147.0,984.0,573.0,1919.0,526,158,1928,974


In [5]:
result.shape

(49, 9)

In [6]:
result.to_csv("yolo_results/samples_test_pred.csv", index = False)

## **Calculating mAP**

In [7]:
# implementation according to: https://datanics.blogspot.com/2020/11/understanding-mean-average-precision.html

# jaccard index
def IoU(df):
    # determining the minimum and maximum -coordinates of the intersection rectangle
    xmin_inter = max(df["xmin"], df["xmin_pred"])
    ymin_inter = max(df["ymin"], df["ymin_pred"])
    xmax_inter = min(df["xmax"], df["xmax_pred"])
    ymax_inter = min(df["ymax"], df["ymax_pred"])
 
    # calculate area of intersection rectangle
    inter_area = max(0, xmax_inter - xmin_inter + 1) * max(0, ymax_inter - ymin_inter + 1)
 
    # calculate area of actual and predicted boxes
    actual_area = (df["xmax"] - df["xmin"] + 1) * (df["ymax"] - df["ymin"] + 1)
    pred_area = (df["xmax_pred"] - df["xmin_pred"] + 1) * (df["ymax_pred"] - df["ymin_pred"] + 1)
 
    # computing intersection over union
    iou = inter_area / float(actual_area + pred_area - inter_area)
 
    # return the intersection over union value
    return iou

result["IoU"] = result.apply(IoU, axis = 1)
result.head()

,NAME,ymin,ymax,xmin,xmax,xmin_pred,ymin_pred,xmax_pred,ymax_pred,IoU
0,002094496.jpg,266.0,1080.0,887.0,1686.0,912,278,1681,1025,0.883374
1,007055006.jpg,100.0,274.0,282.0,429.0,278,102,431,265,0.902842
2,007349675.jpg,226.0,437.0,87.0,361.0,100,219,368,438,0.896797
3,011151221.jpg,82.0,349.0,239.0,429.0,222,83,389,346,0.716000
4,018050712.jpg,147.0,984.0,573.0,1919.0,526,158,1928,974,0.936963


In [8]:
# creating column 'TP/FP' which will store TP for True positive and FP for False positive
# if IOU is greater than 0.5 then TP else FP
result["TP/FP"] = result["IoU"].apply(lambda x: "TP" if x >= 0.5 else "FP")
result.head()

,NAME,ymin,ymax,xmin,xmax,xmin_pred,ymin_pred,xmax_pred,ymax_pred,IoU,TP/FP
0,002094496.jpg,266.0,1080.0,887.0,1686.0,912,278,1681,1025,0.883374,TP
1,007055006.jpg,100.0,274.0,282.0,429.0,278,102,431,265,0.902842,TP
2,007349675.jpg,226.0,437.0,87.0,361.0,100,219,368,438,0.896797,TP
3,011151221.jpg,82.0,349.0,239.0,429.0,222,83,389,346,0.716000,TP
4,018050712.jpg,147.0,984.0,573.0,1919.0,526,158,1928,974,0.936963,TP


In [9]:
# calculating Precision and recall
Precision = []
Recall = []

TP = FP = 0
FN = len(result["TP/FP"] == "TP")
for index , row in result.iterrows():     
    if row["IoU"] > 0.5:
        TP = TP + 1
    else:
        FP = FP + 1    

    try:
        AP = TP / (TP + FP)
        Rec = TP / (TP + FN)
    except ZeroDivisionError:
        AP = Recall = 0.0
    
    Precision.append(AP)
    Recall.append(Rec)


result["Precision"] = Precision
result["Recall"] = Recall

result.head()

,NAME,ymin,ymax,xmin,xmax,xmin_pred,ymin_pred,xmax_pred,ymax_pred,IoU,TP/FP,Precision,Recall
0,002094496.jpg,266.0,1080.0,887.0,1686.0,912,278,1681,1025,0.883374,TP,1.0,0.020000
1,007055006.jpg,100.0,274.0,282.0,429.0,278,102,431,265,0.902842,TP,1.0,0.039216
2,007349675.jpg,226.0,437.0,87.0,361.0,100,219,368,438,0.896797,TP,1.0,0.057692
3,011151221.jpg,82.0,349.0,239.0,429.0,222,83,389,346,0.716000,TP,1.0,0.075472
4,018050712.jpg,147.0,984.0,573.0,1919.0,526,158,1928,974,0.936963,TP,1.0,0.092593


In [10]:
#calculating Interpolated Precision
result["Precision_interpol"] = result.groupby("Recall")["Precision"].transform("max")
result.head()

,NAME,ymin,ymax,xmin,xmax,xmin_pred,ymin_pred,xmax_pred,ymax_pred,IoU,TP/FP,Precision,Recall,Precision_interpol
0,002094496.jpg,266.0,1080.0,887.0,1686.0,912,278,1681,1025,0.883374,TP,1.0,0.020000,1.0
1,007055006.jpg,100.0,274.0,282.0,429.0,278,102,431,265,0.902842,TP,1.0,0.039216,1.0
2,007349675.jpg,226.0,437.0,87.0,361.0,100,219,368,438,0.896797,TP,1.0,0.057692,1.0
3,011151221.jpg,82.0,349.0,239.0,429.0,222,83,389,346,0.716000,TP,1.0,0.075472,1.0
4,018050712.jpg,147.0,984.0,573.0,1919.0,526,158,1928,974,0.936963,TP,1.0,0.092593,1.0


In [11]:
prec_at_rec = []

for recall_level in np.linspace(0.0, 1.0, 11):
    try:
        x = result[result["Recall"] >= recall_level]["Precision"]
        prec = max(x)
    except:
        prec = 0.0
    prec_at_rec.append(prec)
avg_prec = np.mean(prec_at_rec)
print("11 point precision is ", prec_at_rec)
print("mAP is ", avg_prec)

11 point precision is  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mAP is  0.5454545454545454


In [12]:
result.to_csv("yolo_results/samples_test_pred2.csv", index = False)